In [196]:
from attributes import *
import polars as pl
import altair as alt

df = pl.read_csv(
    "data/3_research_ethics_education.csv",
)

In [197]:
def count_y(name):
    return df.group_by(pl.col(name).alias("Y")).len().rename({"len":name})   

def count_list_y(columns, select):
    df = pl.DataFrame({"Y":select})
    for col in columns:
        count = df.group_by(pl.col(col).alias("Y")).len().rename({"len":col})
        df = df.join(count, on="Y", how="left")
    return df

def draw_bar_plot(df, x, y, title, MAP, width=400):

    map = MAP
    if type(MAP) is dict:
        map = MAP.values()

    bar_x = alt.X(x, title=None, axis=None)
    bar_x = bar_x if MAP is None else bar_x.sort(map)

    txt_x = alt.X(x, sort=map) if MAP is not None else alt.X(x)

    bar = df.plot.bar().encode(
        x=bar_x,
        y=alt.Y(y, title=None),
        color=alt.Color(x, title=None, scale=alt.Scale(scheme="tableau20")).sort(map),
    ).properties(
        title=title,
        width=width,
    )

    text = bar.mark_text(
        align="center",
        baseline="bottom",
    ).encode(
        x=txt_x,
        y=y,
        text=y,
        color=alt.value("black"),
    )
    return bar + text

In [198]:
# 
# 가 연구자의 연구윤리 교육 경험
# 

# 연구윤리 규범 범위의 확대 인식 여부
columns = {
    "Q6": "연구윤리 규범 범위의 확대 인식 여부",
}

research_ethics_eduction_experience = df.group_by("Q6").len()
research_ethics_eduction_experience.with_columns(
    (pl.col("Q6") / pl.col("Q6").sum()).alias("percentage"),
).write_csv("figure/3/가_연구자의_연구윤리_교육_경험.csv")


bar = draw_bar_plot(research_ethics_eduction_experience.with_columns(pl.col("Q6").replace_strict(MAP_EXIST)), "Q6:N", "len", "연구윤리 규범 범위의 확대 인식 여부", MAP_EXIST)
bar.show()

alt.LayerChart(...)

In [199]:
# 
# 나. 연구윤리 교육 방법
# 

map = ["원내 집합 교육", "원외 집합 교육", "원내 온라인 강의", "원외 온라인 강의", "기타"]

method = df.select([
    pl.col("Q7_1").value_counts().sort().struct[1].alias(map[0]),
    pl.col("Q7_2").value_counts().sort().struct[1].alias(map[1]),
    pl.col("Q7_3").value_counts().sort().struct[1].alias(map[2]),
    pl.col("Q7_4").value_counts().sort().struct[1].alias(map[3]),
    pl.col("Q7_5").value_counts().sort().struct[1].alias(map[4]),
])[1]

method.write_csv("figure/3/나_연구윤리_교육_방법.csv")
print(method)

bar = draw_bar_plot(method.transpose(include_header=True), "column", "column_0", "연구윤리 교육 방법", map)
bar.save("figure/3/나_연구윤리_교육_방법.png")
bar.show()

shape: (1, 5)
┌────────────────┬────────────────┬──────────────────┬──────────────────┬──────┐
│ 원내 집합 교육 ┆ 원외 집합 교육 ┆ 원내 온라인 강의 ┆ 원외 온라인 강의 ┆ 기타 │
│ ---            ┆ ---            ┆ ---              ┆ ---              ┆ ---  │
│ u32            ┆ u32            ┆ u32              ┆ u32              ┆ u32  │
╞════════════════╪════════════════╪══════════════════╪══════════════════╪══════╡
│ 30             ┆ 4              ┆ 113              ┆ 26               ┆ 2    │
└────────────────┴────────────────┴──────────────────┴──────────────────┴──────┘


alt.LayerChart(...)

In [200]:
# 
# 다. 연구윤리 교육 내용
# 

map = ["책임 있는 연구수행", "출판윤리", "연구 공동체", "연구자의 사회적 책임", "연구 성과 및 데이터", "이해충돌", "생명윤리", "연구진실성 검증", "학문교류 윤리", "부실학술활동 예방", "ChatGPT 등 생성형 AI 활용과 연구윤리"]

contents = df.select([
    pl.col("Q8_1").value_counts().sort().struct[1].alias(map[0]),
    pl.col("Q8_2").value_counts().sort().struct[1].alias(map[1]),
    pl.col("Q8_3").value_counts().sort().struct[1].alias(map[2]),
    pl.col("Q8_4").value_counts().sort().struct[1].alias(map[3]),
    pl.col("Q8_5").value_counts().sort().struct[1].alias(map[4]),
    pl.col("Q8_6").value_counts().sort().struct[1].alias(map[5]),
    pl.col("Q8_7").value_counts().sort().struct[1].alias(map[6]),
    pl.col("Q8_8").value_counts().sort().struct[1].alias(map[7]),
    pl.col("Q8_9").value_counts().sort().struct[1].alias(map[8]),
    pl.col("Q8_10").value_counts().sort().struct[1].alias(map[9]),
    pl.col("Q8_11").value_counts().sort().struct[1].alias(map[10]),
])[1]

contents.write_csv("figure/3/다_연구윤리_교육_내용.csv")
print(contents)

bar = draw_bar_plot(contents.transpose(include_header=True), "column", "column_0", "연구윤리 교육 내용", map, width=800)
bar.save("figure/3/다_연구윤리_교육_내용.png")
bar.show()

shape: (1, 11)
┌────────────┬──────────┬────────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ 책임 있는  ┆ 출판윤리 ┆ 연구   ┆ 연구자의   ┆ … ┆ 연구진실성 ┆ 학문교류  ┆ 부실학술  ┆ ChatGPT   │
│ 연구수행   ┆ ---      ┆ 공동체 ┆ 사회적     ┆   ┆ 검증       ┆ 윤리      ┆ 활동 예방 ┆ 등 생성형 │
│ ---        ┆ u32      ┆ ---    ┆ 책임       ┆   ┆ ---        ┆ ---       ┆ ---       ┆ AI 활용과 │
│ u32        ┆          ┆ u32    ┆ ---        ┆   ┆ u32        ┆ u32       ┆ u32       ┆ 연구윤리  │
│            ┆          ┆        ┆ u32        ┆   ┆            ┆           ┆           ┆ ---       │
│            ┆          ┆        ┆            ┆   ┆            ┆           ┆           ┆ u32       │
╞════════════╪══════════╪════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 93         ┆ 93       ┆ 30     ┆ 34         ┆ … ┆ 41         ┆ 34        ┆ 49        ┆ 18        │
└────────────┴──────────┴────────┴────────────┴───┴────────────┴───────────┴───────────┴───────────┘


alt.LayerChart(...)

In [210]:
# 
# 라. 연구윤리 교육 만족도
# 
# 1)
column = {
    "Q9": "연구윤리 교육 만족도",
}

count = df.group_by("Q9").len().drop_nulls().sort("Q9")
all = count.with_columns((pl.col("len") / pl.col("len").sum()).alias("percentage")).rename(column)
all.write_csv("figure/3/라_1_연구윤리_교육_만족도.csv", include_bom=True)

In [215]:
# 
# 라. 연구윤리 교육 만족도
# 
# 2)
column = {
    "Q10": "연구윤리 교육 만족의 이유",
}

count = df.group_by("Q10").len().drop_nulls().sort("Q10")
all = count.with_columns((pl.col("len") / pl.col("len").sum()).alias("percentage")).rename(column)
all.write_csv("figure/3/라_2_연구윤리_교육_만족_이유.csv", include_bom=True)

In [216]:
# 
# 라. 연구윤리 교육 만족도
# 
# 3)
column = {
    "Q11": "연구윤리 교육 불만족의 이유",
}

count = df.group_by("Q11").len().drop_nulls().sort("Q11")
all = count.with_columns((pl.col("len") / pl.col("len").sum()).alias("percentage")).rename(column)
all.write_csv("figure/3/라_2_연구윤리_교육_불만족_이유.csv", include_bom=True)

In [217]:
# 
# 마. 연구자가 향후 받고 싶은 교육 내용
# 

map = ["연구부정행위 사례 및 검증", "출판윤리", "연구비 집행", "연구노트 작성 및 데이터 관리", "연구자의 사회적 책임", "부실 학술활동 예방", "AI 활용 연구활동과 연구윤리 누락", "이해충돌 관리", "기타"]

future_contents = df.select([
    pl.col("Q12_1").value_counts().sort().struct[1].alias(map[0]),
    pl.col("Q12_2").value_counts().sort().struct[1].alias(map[1]),
    pl.col("Q12_3").value_counts().sort().struct[1].alias(map[2]),
    pl.col("Q12_4").value_counts().sort().struct[1].alias(map[3]),
    pl.col("Q12_5").value_counts().sort().struct[1].alias(map[4]),
    pl.col("Q12_6").value_counts().sort().struct[1].alias(map[5]),
    pl.col("Q12_7").value_counts().sort().struct[1].alias(map[6]),
    pl.col("Q12_8").value_counts().sort().struct[1].alias(map[7]),
    pl.col("Q12_9").value_counts().sort().struct[1].alias(map[8]),
])[1]

future_contents.write_csv("figure/3/마_연구자가_향후_받고_싶은_교육_내용.csv", include_bom=True)

bar = draw_bar_plot(future_contents.transpose(include_header=True), "column", "column_0", "연구자가 향후 받고 싶은 교육 내용", map)
bar.save("figure/3/마_연구자가_향후_받고_싶은_교육_내용.svg")

shape: (1, 9)
┌─────────────┬──────────┬────────┬─────────────┬───┬─────────────┬────────────┬────────────┬──────┐
│ 연구부정행  ┆ 출판윤리 ┆ 연구비 ┆ 연구노트    ┆ … ┆ 부실        ┆ AI 활용    ┆ 이해충돌   ┆ 기타 │
│ 위 사례 및  ┆ ---      ┆ 집행   ┆ 작성 및     ┆   ┆ 학술활동    ┆ 연구활동과 ┆ 관리       ┆ ---  │
│ 검증        ┆ u32      ┆ ---    ┆ 데이터 관리 ┆   ┆ 예방        ┆ 연구윤리   ┆ ---        ┆ u32  │
│ ---         ┆          ┆ u32    ┆ ---         ┆   ┆ ---         ┆ 누락       ┆ u32        ┆      │
│ u32         ┆          ┆        ┆ u32         ┆   ┆ u32         ┆ ---        ┆            ┆      │
│             ┆          ┆        ┆             ┆   ┆             ┆ u32        ┆            ┆      │
╞═════════════╪══════════╪════════╪═════════════╪═══╪═════════════╪════════════╪════════════╪══════╡
│ 93          ┆ 73       ┆ 75     ┆ 35          ┆ … ┆ 35          ┆ 93         ┆ 33         ┆ 1    │
└─────────────┴──────────┴────────┴─────────────┴───┴─────────────┴────────────┴────────────┴──────┘


alt.LayerChart(...)

In [219]:
# 
# 바. 연구자가 향후 받고 싶은 교육 방법
# 

map = ["대면", "비대면", "사례중심", "토의/토론 중심", "강의 중심", "동영상 등 ICT 활용", "기타"]

future_method = df.select([
    pl.col("Q13_1").value_counts().sort().struct[1].alias(map[0]),
    pl.col("Q13_2").value_counts().sort().struct[1].alias(map[1]),
    pl.col("Q13_3").value_counts().sort().struct[1].alias(map[2]),
    pl.col("Q13_4").value_counts().sort().struct[1].alias(map[3]),
    pl.col("Q13_5").value_counts().sort().struct[1].alias(map[4]),
    pl.col("Q13_6").value_counts().sort().struct[1].alias(map[5]),
    # pl.col("Q13_7").value_counts().sort().struct[1].alias(map[6]),
])[1]

future_method.write_csv("figure/3/바_연구자가_향후_받고_싶은_교육_방법.csv", include_bom=True)

bar = draw_bar_plot(future_method.transpose(include_header=True), "column", "column_0", "연구자가 향후 받고 싶은 교육 방법", map)
bar.save("figure/3/바_연구자가_향후_받고_싶은_교육_방법.svg")

In [187]:
# 
# 사. KIOST가 제공하는 연구윤리 교육 콘텐츠
# 
# 1) KIOST 연구윤리 교육 콘텐츠 확인 여부
# 

map = {
    1: "보지 못했다",
    2: "보았다"
}

video = df.group_by("Q14_1").len().rename({"Q14_1": "확인여부"}).with_columns(구분=pl.lit("연구윤리 동영상"))
newsletter = df.group_by("Q14_2").len().rename({"Q14_2": "확인여부"}).with_columns(구분=pl.lit("연구윤리 뉴스레터"))
qanda = df.group_by("Q14_3").len().rename({"Q14_3": "확인여부"}).with_columns(구분=pl.lit("연구윤리 Q&A"))
checklist = df.group_by("Q14_4").len().rename({"Q14_4": "확인여부"}).with_columns(구분=pl.lit("연구윤리 체크리스트"))


KIOST_contents = pl.DataFrame()
KIOST_contents = KIOST_contents.vstack(video).vstack(newsletter).vstack(qanda).vstack(checklist).with_columns(
    pl.col("확인여부").replace_strict(map),
)
KIOST_contents.write_csv("figure/3/사_1_KIOST_연구윤리_교육_콘텐츠_확인_여부.csv", include_bom=True)

bar = KIOST_contents.plot.bar().encode(
    x=alt.X("확인여부:O", title=None, axis=None),
    y=alt.Y("len:Q", title=None),
    color=alt.Color("확인여부:N"),
    # column=alt.Column("구분:N", title="KIOST 연구윤리 교육 콘텐츠 확인 여부", center=True, align="each"),
).properties(
    width=120,
)

text = bar.mark_text(
    align="center",
    baseline="bottom",
).encode(
    text="len",
)

layered_chart = alt.layer(bar, text)

# Apply faceting to the layered chart
final_chart = layered_chart.facet(
    column=alt.Column("구분:N", title="KIOST 연구윤리 교육 콘텐츠 확인 여부")
)

# Display the final chart
final_chart.display()

shape: (8, 3)
┌─────────────┬─────┬─────────────────────┐
│ 확인여부    ┆ len ┆ 구분                │
│ ---         ┆ --- ┆ ---                 │
│ str         ┆ u32 ┆ str                 │
╞═════════════╪═════╪═════════════════════╡
│ 보았다      ┆ 110 ┆ 연구윤리 동영상     │
│ 보지 못했다 ┆ 57  ┆ 연구윤리 동영상     │
│ 보지 못했다 ┆ 41  ┆ 연구윤리 뉴스레터   │
│ 보았다      ┆ 126 ┆ 연구윤리 뉴스레터   │
│ 보지 못했다 ┆ 69  ┆ 연구윤리 Q&A        │
│ 보았다      ┆ 98  ┆ 연구윤리 Q&A        │
│ 보았다      ┆ 82  ┆ 연구윤리 체크리스트 │
│ 보지 못했다 ┆ 85  ┆ 연구윤리 체크리스트 │
└─────────────┴─────┴─────────────────────┘


alt.FacetChart(...)

In [188]:
# 
# 사. KIOST가 제공하는 연구윤리 교육 콘텐츠
# 
# 2) KIOST 연구윤리 교육 콘텐츠 만족도
# 

map = {
    1: "매우 불만족",
    2: "불만족",
    3: "보통",
    4: "만족",
    5: "매우 만족"
}

video = df.group_by("Q15_1").len().rename({"Q15_1": "만족도"}).with_columns(구분=pl.lit("연구윤리 동영상")).drop_nulls()
newsletter = df.group_by("Q15_2").len().rename({"Q15_2": "만족도"}).with_columns(구분=pl.lit("연구윤리 뉴스레터")).drop_nulls()
qanda = df.group_by("Q15_3").len().rename({"Q15_3": "만족도"}).with_columns(구분=pl.lit("연구윤리 Q&A")).drop_nulls()
checklist = df.group_by("Q15_4").len().rename({"Q15_4": "만족도"}).with_columns(구분=pl.lit("연구윤리 체크리스트")).drop_nulls()

KIOST_contents = pl.DataFrame()
KIOST_contents = KIOST_contents.vstack(video).vstack(newsletter).vstack(qanda).vstack(checklist).with_columns(
    pl.col("만족도").replace_strict(map),
)

print(KIOST_contents)

bar = KIOST_contents.plot.bar().encode(
    x=alt.X("만족도:O", title=None, axis=None),
    y=alt.Y("len:Q", title=None),
    color=alt.Color("만족도:N"),
    # column=alt.Column("구분:N", title="KIOST 연구윤리 교육 콘텐츠 확인 여부", center=True, align="each"),
).properties(
    width=120,
)

text = bar.mark_text(
    align="center",
    baseline="bottom",
).encode(
    text="len",
)

layered_chart = alt.layer(bar, text)

# Apply faceting to the layered chart
final_chart = layered_chart.facet(
    column=alt.Column("구분:N", title="KIOST 연구윤리 교육 콘텐츠 확인 여부")
)

# Display the final chart
final_chart.display()

shape: (16, 3)
┌─────────────┬─────┬─────────────────────┐
│ 만족도      ┆ len ┆ 구분                │
│ ---         ┆ --- ┆ ---                 │
│ str         ┆ u32 ┆ str                 │
╞═════════════╪═════╪═════════════════════╡
│ 보통        ┆ 33  ┆ 연구윤리 동영상     │
│ 만족        ┆ 3   ┆ 연구윤리 동영상     │
│ 매우 불만족 ┆ 18  ┆ 연구윤리 동영상     │
│ 불만족      ┆ 60  ┆ 연구윤리 동영상     │
│ 불만족      ┆ 70  ┆ 연구윤리 뉴스레터   │
│ …           ┆ …   ┆ …                   │
│ 매우 불만족 ┆ 25  ┆ 연구윤리 Q&A        │
│ 불만족      ┆ 45  ┆ 연구윤리 체크리스트 │
│ 만족        ┆ 3   ┆ 연구윤리 체크리스트 │
│ 매우 불만족 ┆ 20  ┆ 연구윤리 체크리스트 │
│ 보통        ┆ 28  ┆ 연구윤리 체크리스트 │
└─────────────┴─────┴─────────────────────┘


alt.FacetChart(...)

In [189]:
# 
# 3) KIOST 연구윤리 교육 콘텐츠별 수정 및 보완 요구 사항
# 



In [190]:
# 
# 4) 향후 지속적 개발 제공이 필요한 연구윤리 교육 콘텐츠
# 

KIOST_contents_to_improve = df.select([
    pl.col("Q17_1").value_counts().sort().struct[1].alias("연구윤리 동영상"),
    pl.col("Q17_2").value_counts().sort().struct[1].alias("연구윤리 뉴스레터"),
    pl.col("Q17_3").value_counts().sort().struct[1].alias("연구윤리 Q&A"),
    pl.col("Q17_4").value_counts().sort().struct[1].alias("연구윤리 체크리스트"),
    pl.col("Q17_5").value_counts().sort().struct[1].alias("기타"),
])[1]

KIOST_contents_to_improve.write_csv("figure/3/사_4_향후_지속적_개발_제공이_필요한_연구윤리_교육_콘텐츠.csv")
print(KIOST_contents_to_improve)

bar = draw_bar_plot(KIOST_contents_to_improve.transpose(include_header=True), "column", "column_0", "향후 지속적 개발 제공이 필요한 연구윤리 교육 콘텐츠", ["연구윤리 동영상", "연구윤리 뉴스레터", "연구윤리 Q&A", "연구윤리 체크리스트", "기타"])
bar.save("figure/3/사_4_향후_지속적_개발_제공이_필요한_연구윤리_교육_콘텐츠.png")
bar.show()

shape: (1, 5)
┌─────────────────┬───────────────────┬──────────────┬─────────────────────┬──────┐
│ 연구윤리 동영상 ┆ 연구윤리 뉴스레터 ┆ 연구윤리 Q&A ┆ 연구윤리 체크리스트 ┆ 기타 │
│ ---             ┆ ---               ┆ ---          ┆ ---                 ┆ ---  │
│ u32             ┆ u32               ┆ u32          ┆ u32                 ┆ u32  │
╞═════════════════╪═══════════════════╪══════════════╪═════════════════════╪══════╡
│ 60              ┆ 74                ┆ 74           ┆ 74                  ┆ 3    │
└─────────────────┴───────────────────┴──────────────┴─────────────────────┴──────┘


alt.LayerChart(...)

In [191]:
# 
# 5) 연구윤리 Q&A 사례집에 대한 보완 사항
# 



In [192]:
# 
# 아. 연구윤리 컨설팅 희망 내용
# 

In [193]:
# 
# chi sqaure
# 
